In [ ]:
! sudo apt-get install pigz 

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  pigz
0 upgraded, 1 newly installed, 0 to remove and 20 not upgraded.
Need to get 57.4 kB of archives.
After this operation, 259 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 pigz amd64 2.4-1 [57.4 kB]
Fetched 57.4 kB in 0s (987 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: un

In [ ]:
!nvidia-smi

Wed Aug 31 04:51:17 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#Data

In [ ]:
!pip install kaggle
from google.colab import files
from datetime import datetime
api_token = files.upload()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Saving kaggle.json to kaggle.json


In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle competitions download -c 'cdiscount-image-classification-challenge' -f train.bson

100% 47.2G/47.2G [04:37<00:00, 336MB/s]
100% 47.2G/47.2G [04:37<00:00, 183MB/s]


In [ ]:
!kaggle competitions download -c 'cdiscount-image-classification-challenge' -f category_names.csv

  0% 0.00/406k [00:00<?, ?B/s]
100% 406k/406k [00:00<00:00, 93.5MB/s]


In [ ]:
!unzip /content/train.bson.zip

Archive:  /content/train.bson.zip
  inflating: train.bson              


In [ ]:
!rm /content/train.bson.zip

In [ ]:
import bson
import numpy as np
import pandas as pd
import os
from tqdm import tqdm_notebook
from random import random
import pickle as pkl
from collections import defaultdict

In [ ]:
out_folder_train = '/content/train'
#out_folder_validation = '/content/drive/Shareddrives/neil_rathod/cs2/Validation'
out_folder_validation = '/content/validation'
out_folder_test = '/content/test'

# Create output folders
if not os.path.exists(out_folder_train):
    os.makedirs(out_folder_train)
    
if not os.path.exists(out_folder_test):
    os.makedirs(out_folder_test)

if not os.path.exists(out_folder_validation):
    os.makedirs(out_folder_validation)

In [ ]:
# Create categories folders
categories = pd.read_csv('/content/drive/Shareddrives/Case_study/cdiscount/category_names.csv', index_col='category_id')

In [ ]:
for category in tqdm_notebook(categories.index):
    os.mkdir(os.path.join(out_folder_train, str(category)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


  0%|          | 0/5270 [00:00<?, ?it/s]

In [ ]:
for category in tqdm_notebook(categories.index):
    os.mkdir(os.path.join(out_folder_validation, str(category)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


  0%|          | 0/5270 [00:00<?, ?it/s]

In [ ]:
num_product_total = 7069896
num_products_train = int(num_product_total*0.80)  
num_product_validation = num_product_total - num_products_train

bar = tqdm_notebook(total=num_product_total)
cnt_train = 0
cnt_val = 0
validation_class_to_images = defaultdict(list)
train_class_to_images = defaultdict(list)
with open('/content/drive/Shareddrives/Case_study/cdiscount/train.bson', 'rb') as fbson:

    data = bson.decode_file_iter(fbson)
    
    for c, d in enumerate(data):
        category = d['category_id']
        _id = d['_id']
        for e, pic in enumerate(d['imgs']):
            if random() <= 0.80 :
                fname = os.path.join(out_folder_train, str(category), '{}-{}.jpg'.format(_id, e))
                train_class_to_images[category].append(fname)
                cnt_train +=1
            else:
                fname = os.path.join(out_folder_validation, str(category), '{}-{}.jpg'.format(_id, e))
                validation_class_to_images[category].append(fname)
                cnt_val +=1
                
            with open(fname, 'wb') as f:
                f.write(pic['picture'])

        bar.update()

print("Total number of images : {}\n \
      Number of train example : {} ({}%)\n \
      Number of validation examples : {} ({}%)".format((cnt_train+cnt_val), cnt_train, cnt_train/(cnt_train+cnt_val), cnt_val, cnt_val/(cnt_train+cnt_val)))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/7069896 [00:00<?, ?it/s]

Total number of images : 12371293
       Number of train example : 11133762 (0.8999675296672708%)
       Number of validation examples : 1237531 (0.10003247033272916%)


In [ ]:
! sudo apt-get install pigz

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  pigz
0 upgraded, 1 newly installed, 0 to remove and 20 not upgraded.
Need to get 57.4 kB of archives.
After this operation, 259 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 pigz amd64 2.4-1 [57.4 kB]
Fetched 57.4 kB in 1s (60.6 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: u

In [ ]:
!tar cf - /content/train | pigz > train90.tar.gz

tar: Removing leading `/' from member names


In [ ]:
import shutil
  
# Source path 100006949  
source = "/content/train80.tar.gz"
  
# Destination path
destination = "/content/drive/Shareddrives/datascience/exp/train80.tar.gz"
  
# Move the content of
# source to destination
dest = shutil.move(source, destination)
print('done')

done


In [ ]:
!pigz -dc /content/drive/Shareddrives/datascience/exp/train80.tar.gz | tar xf -

#Model

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import io
import keras
import bson                       # this is installed with the pymongo package
from imageio import imread  # or, whatever image library you prefer
import multiprocessing as mp      # will come in handy due to the size of the data
import time
import datetime as dt

import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
import concurrent.futures
from multiprocessing import cpu_count



from keras import layers
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, Dropout, Flatten, merge, Reshape, Activation
from keras.layers import GlobalAveragePooling2D
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D, GlobalAveragePooling2D, GlobalMaxPooling2D
from keras.layers import BatchNormalization
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras import backend as K
from keras.utils.data_utils import get_file
import tensorflow as tf
from sklearn.metrics import log_loss
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "/content/drive/Shareddrives/Case_study/cdiscount"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

category_names.csv
inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
train.bson
train_example.bson
train_offsets.csv
train.tar.gz
Untitled0.ipynb
validation.tar.gz



In [ ]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.callbacks import Callback
from keras.callbacks import TensorBoard
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import LearningRateScheduler

In [ ]:
from keras.applications.xception import Xception
from keras.applications.xception import preprocess_input

#working

In [ ]:
def lr_scheduler(epoch, lr):
    'For every 3rd epoch, decay learning rate by 5%'
    decay_rate = .97
    decay_step = 3
    if (epoch+1) % decay_step == 0 :
        return lr * decay_rate
    return lr


In [ ]:
#import datetime, os
# Model is saved to this location
#path = os.path.abspath('/content/drive/Shareddrives/datascience/exp/model_data/')
#os.mkdir(os.path.join(path, '4thIter'))
#path = os.path.join(path, '4thIter')

In [15]:
#from keras.callbacks import ModelCheckpoint, CSVLogger
#import datetime, os

#filepath="/content/drive/Shareddrives/datascience/exp/model_data/weights-{epoch:02d}-{val_accuracy:.4f}.hdf5"
#reduce learning rate by 10% based on validation accuracy

reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.1, patience=1, min_lr=0)

#ckpt = ModelCheckpoint(os.path.join(path, 'model.hdf5'), monitor = 'val_accuracy',save_freq=1)

#csv = CSVLogger(os.path.join(path, 'log.csv'), append = True)

lrschedule = LearningRateScheduler(lr_scheduler, verbose=0)

#logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

checkpoints = [reduce_lr,lrschedule]

In [ ]:
INPUT_PATH  = '/content/drive/Shareddrives/Case_study/cdiscount'
train_data_dir      = '/content/content/train'

img_width  = 180
img_height = 180
batch_size = 500

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
    validation_split=0.05) # set validation split
# only rescaling
print('Time right now: ', dt.datetime.now())
start = time.time()
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=False) # set as training data

end = time.time()
print('Time right now: ', dt.datetime.now())
print('It took ', end-start, ' seconds to prepare train gen')
start = time.time()

validation_generator = train_datagen.flow_from_directory(
    train_data_dir, # same directory as training data
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False) # set as validation data
end = time.time()

print('It took ', end-start, ' seconds to prepare valid gen')

Time right now:  2022-09-01 04:39:57.493065
Found 9405303 images belonging to 5270 classes.
Time right now:  2022-09-01 04:44:56.234417
It took  298.741060256958  seconds to prepare train gen
Found 492390 images belonging to 5270 classes.
It took  149.12905144691467  seconds to prepare valid gen


In [ ]:
IMAGE_SIZE = [180, 180] #pre trained Xception model
model = Xception(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

83697664/83683744 [==============================] - 1s 0us/step


In [ ]:
#model_1
for layer in model.layers:
  layer.trainable = False
#Adding custom Layers 
x = model.output
x = Conv2D(filters=512,kernel_size=(3,3),padding="same", activation="relu")(x)
x = MaxPool2D(2,2)(x)
x = Flatten()(x)
x = Dense(500, activation="relu")(x)
x = Dense(500, activation="relu")(x)
output = Dense(5270, activation="softmax")(x)
# creating the final model 
model_1 = Model(inputs = model.input, outputs = output)
# compile the model 
Adam = tf.keras.optimizers.Adam(lr=0.0003, beta_1=0.9, beta_2=0.999, epsilon=1e-07,amsgrad=False,name='Adam')
model_1.compile(loss = "categorical_crossentropy", optimizer = Adam, metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
model_1.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 180, 180, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 89, 89, 32)   864         ['input_1[0][0]']                
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 89, 89, 32)  128         ['block1_conv1[0][0]']           
 ation)                                                                                           
                                                                                              

In [ ]:
# model check points causing more training time 

In [ ]:
nb_epochs=3
model_1.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = nb_epochs,
    workers = 8)
    #callbacks=checkpoints)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/3
31351/31351 [==============================] - 15548s 495ms/step - loss: 5.3195 - accuracy: 0.2470 - val_loss: 4.2552 - val_accuracy: 0.3448
Epoch 2/3
31351/31351 [==============================] - 15335s 489ms/step - loss: 4.1315 - accuracy: 0.3732 - val_loss: 3.8256 - val_accuracy: 0.3976
Epoch 3/3
31351/31351 [==============================] - 15314s 488ms/step - loss: 3.8113 - accuracy: 0.4123 - val_loss: 3.6341 - val_accuracy: 0.4255


In [ ]:
model_1.save_weights('/content/drive/Shareddrives/datascience/exp/saved_model_weights/3epochs.h5')
print("Wights Saved")

Wights Saved


In [ ]:
model_1.save('/content/drive/Shareddrives/datascience/exp/saved_model/3epoch_model.h5')
print("Model Saved")

Model Saved


In [ ]:
from keras.models import load_model
demo = load_model('/content/drive/Shareddrives/datascience/exp/saved_model/3epoch_model.h5')

In [ ]:
nb_epochs=1
demo.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = nb_epochs,
    workers = 8)
    #callbacks=checkpoints)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


31351/31351 [==============================] - 15421s 492ms/step - loss: 3.6183 - accuracy: 0.4355 - val_loss: 3.4986 - val_accuracy: 0.4431


In [ ]:
model_1.save_weights('/content/drive/Shareddrives/datascience/exp/saved_model_weights/4epochs.h5')
print("Wights Saved")

Wights Saved


In [ ]:
model_1.save('/content/drive/Shareddrives/datascience/exp/saved_model/4epoch_model.h5')
print("Model Saved")

Model Saved


increased batch size from 300 to 500 and decreased lr rate from 0.0003 to 0.0002

In [ ]:
from keras.models import load_model
demo = load_model('/content/drive/Shareddrives/datascience/exp/saved_model/4epoch_model.h5')

In [ ]:
#keras.backend.set_value(model.optimizer.lr, .0002)

In [ ]:
nb_epochs = 3
model_1.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = nb_epochs,
    workers = 8)
    #callbacks=checkpoints)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/3
18810/18810 [==============================] - 15532s 826ms/step - loss: 3.4561 - accuracy: 0.4454 - val_loss: 3.4596 - val_accuracy: 0.4475
Epoch 2/3
18810/18810 [==============================] - 15661s 832ms/step - loss: 3.3842 - accuracy: 0.4525 - val_loss: 3.4136 - val_accuracy: 0.4523
Epoch 3/3
18810/18810 [==============================] - 15606s 830ms/step - loss: 3.3291 - accuracy: 0.4575 - val_loss: 3.3765 - val_accuracy: 0.4554


In [ ]:
model_1.save_weights('/content/drive/Shareddrives/datascience/exp/saved_model_weights/7epochs.h5')
print("Wights Saved")

Wights Saved


In [ ]:
model_1.save('/content/drive/Shareddrives/datascience/exp/saved_model/7epoch_model.h5')
print("Model Saved")

Model Saved


In [ ]:
from keras.models import load_model
demo = load_model('/content/drive/Shareddrives/datascience/exp/saved_model/7epoch_model.h5')

In [ ]:
keras.backend.set_value(demo.optimizer.lr, 0.00003)

In [ ]:
nb_epochs = 1
demo.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = nb_epochs,
    workers = 8)
    #callbacks=checkpoints)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


18810/18810 [==============================] - 15648s 832ms/step - loss: 3.2789 - accuracy: 0.4617 - val_loss: 3.3394 - val_accuracy: 0.4588


In [ ]:
demo.save_weights('/content/drive/Shareddrives/datascience/exp/saved_model_weights/8epochs.h5')
print("Wights Saved")

Wights Saved


In [ ]:
demo.save('/content/drive/Shareddrives/datascience/exp/saved_model/8epoch_model.h5')
print("Model Saved")

Model Saved


In [ ]:
from keras.models import load_model
demo = load_model('/content/drive/Shareddrives/datascience/exp/saved_model/8epoch_model.h5')

In [ ]:
nb_epochs = 4
demo.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = nb_epochs,
    workers = 8)
    #callbacks=checkpoints)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/4
18810/18810 [==============================] - 15609s 829ms/step - loss: 3.2323 - accuracy: 0.4657 - val_loss: 3.3106 - val_accuracy: 0.4613
Epoch 2/4
18810/18810 [==============================] - 15583s 828ms/step - loss: 3.1911 - accuracy: 0.4691 - val_loss: 3.2752 - val_accuracy: 0.4639
Epoch 3/4
18810/18810 [==============================] - 15575s 828ms/step - loss: 3.1507 - accuracy: 0.4726 - val_loss: 3.2495 - val_accuracy: 0.4664
Epoch 4/4
18421/18810 [============================>.] - ETA: 5:05 - loss: 3.1128 - accuracy: 0.4755

In [ ]:
demo.save_weights('/content/drive/Shareddrives/datascience/exp/saved_model_weights/12epochs.h5')
print("Wights Saved")

In [ ]:
demo.save('/content/drive/Shareddrives/datascience/exp/saved_model/12epoch_model.h5')
print("Model Saved")

In [ ]:
from keras.models import load_model
demo = load_model('/content/drive/Shareddrives/datascience/exp/saved_model/12epoch_model.h5')

In [ ]:
#keras.backend.set_value(demo.optimizer.lr, .000003)

In [11]:
nb_epochs = 3
demo.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = nb_epochs,
    workers = 8,
    callbacks=[reduce_lr,lrschedule])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  if __name__ == '__main__':


Epoch 1/3
18810/18810 [==============================] - 15474s 822ms/step - loss: 3.0735 - accuracy: 0.4794 - val_loss: 3.1964 - val_accuracy: 0.4712 - lr: 3.0000e-05
Epoch 2/3
18810/18810 [==============================] - 15101s 803ms/step - loss: 3.0379 - accuracy: 0.4826 - val_loss: 3.1758 - val_accuracy: 0.4746 - lr: 3.0000e-05
Epoch 3/3
18810/18810 [==============================] - 14826s 788ms/step - loss: 3.0031 - accuracy: 0.4863 - val_loss: 3.1514 - val_accuracy: 0.4768 - lr: 2.9100e-05


In [13]:
demo.save_weights('/content/drive/Shareddrives/datascience/exp/saved_model_weights/15epochs.h5')
print("Wights Saved")

Wights Saved


In [14]:
demo.save('/content/drive/Shareddrives/datascience/exp/saved_model/15epoch_model.h5',include_optimizer=True)
print("Model Saved")

Model Saved


In [17]:
keras.backend.set_value(demo.optimizer.lr, 0.000003)

In [18]:
nb_epochs = 2
demo.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = nb_epochs,
    workers = 8,
    callbacks=[reduce_lr,lrschedule])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  if __name__ == '__main__':


Epoch 1/2
18810/18810 [==============================] - ETA: 0s - loss: 2.9423 - accuracy: 0.4920

18810/18810 [==============================] - 15068s 801ms/step - loss: 2.9423 - accuracy: 0.4920 - val_loss: 3.1437 - val_accuracy: 0.4780 - lr: 3.0000e-06
Epoch 2/2
18810/18810 [==============================] - ETA: 0s - loss: 2.9345 - accuracy: 0.4929

18810/18810 [==============================] - 15218s 809ms/step - loss: 2.9345 - accuracy: 0.4929 - val_loss: 3.1411 - val_accuracy: 0.4783 - lr: 3.0000e-06


In [19]:
demo.save('/content/drive/Shareddrives/datascience/exp/saved_model/17epoch_model.h5',include_optimizer=True)
print("Model Saved")

Model Saved
